# 0. 참고자료
## 0-1. 도서
- 비제이퍼블릭 | 한주현 저 - 바이오파이썬으로 만나는 생물정보학

## 0-2. 웹사이트
- biopython SeqIO | [biopython wiki SeqIO](https://biopython.org/wiki/SeqIO)

## 0-999. 데이터셋 출처
- 도서 깃헙 | [바이오 파이썬으로 만나는 생물정보학 깃헙](https://github.com/bjpublic/biopython)

In [4]:
## 아래 반복문을 통해 나온 결과들을 예쁘게 나오도록하는 씨잘데기 없는 함수
def string_decoration(string, num = 80, idx=1):
    decorated_string = f'{"-"*num}\n{string}\n{"-"*num}' if idx == 1 else f'{string} \n{"-"*num}'
    print(decorated_string)

string_decoration('아아 함수 테스트')
string_decoration('잘 나옵니까', idx=2)

--------------------------------------------------------------------------------
아아 함수 테스트
--------------------------------------------------------------------------------
잘 나옵니까 
--------------------------------------------------------------------------------


# 1.SeqRecord 객체
- biopython의 Seq 객체에 서열 뿐만 아니라, 서열의 이름, NCBI의 ID 등 정보들이 들어있는 객체
- 서열 정보를 제외한 다른 정보는 기본값으로 unknown이 들어간다. (서열 정보는 반드시 들어가야 한다.)

|  속성이름   |     설명   |
|:----------:|:----------|
|seq              |서열 정보를 의미                                                   |
|id               |Locus와 같은 id 정보를 의미                                        |
|name             |서열의 의미를 의미하며 유전자 이름과 같은 정보가 들어감                |
|description      |추가적인 설명이 들어있음                                            |
|letter_annotation|파이썬 딕셔너리 형으로 사용자가 추가 설명을 키, 값 쌍으로 입력할 수 있음|
|annotations      |추가 설명이 파이썬 딕셔너리 형으로 들어감                             |
|features         |서열 구간에 대한 특징이 들어감                                       |
|dbxrefs          |추가 db가 있다면 내용이 들어감                                       |

## 1-1. SeqReocrd 객체 생성

In [1]:
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

dummy_seq = Seq('AGTAAGGGTTAGGAATA')
dummy_seq_record = SeqRecord(dummy_seq)

print(f'원본 Seq  객체 : {dummy_seq}\n')
print(f'SeqRecord 객체 \n{dummy_seq_record}\n')

try:
    error_test = SeqRecord()
except Exception as e:
    print(f'SeqRecord 함수에 Seq를 입력하지 않은 경우 : {e}')

원본 Seq  객체 : AGTAAGGGTTAGGAATA

SeqRecord 객체 
ID: <unknown id>
Name: <unknown name>
Description: <unknown description>
Number of features: 0
Seq('AGTAAGGGTTAGGAATA')

SeqRecord 함수에 Seq를 입력하지 않은 경우 : __init__() missing 1 required positional argument: 'seq'


### (!) SeqRecord 객체에 정보를 넣는 방법은 두 가지 있다.
- SeqRecord 객체를 생성할 때 입력하기
- SeqRecord의 속성에 접근하여 입력하기
   - 이와 같은 방법으로 속성 값을 변경할 수도 있다

In [90]:
## SeqRecord 객체에 설명 입력하기
dummy_seq_record = SeqRecord(dummy_seq, id = 'dummy_9999', name = 'dummy')
print(f'SeqRecord 객체 생성시 정보 입력하여 테스트 \n{dummy_seq_record}\n')

dummy_seq_record.name = 'new name'
dummy_seq_record.description = 'sequence name change'
dummy_seq_record.annotations['dummy1'] = 'dummy1'
dummy_seq_record.letter_annotion = {'dummy2' : 'dummy2'}
print(f'SeqRecord 객체의 속성에 접근하여 테스트 \n{dummy_seq_record}\n')

SeqRecord 객체 생성시 정보 입력하여 테스트 
ID: dummy_9999
Name: dummy
Description: <unknown description>
Number of features: 0
Seq('AGTAAGGGTTAGGAATA')

SeqRecord 객체의 속성에 접근하여 테스트 
ID: dummy_9999
Name: new name
Description: sequence name change
Number of features: 0
/dummy1=dummy1
Seq('AGTAAGGGTTAGGAATA')



## 1-2.SeqRecord 객체 간 비교하기
- SeqRecord 객체는 두 객체간 직접적인 비교는 불가능하고, 객체 안 속성 값을 비교해야한다.

In [222]:
dummy_seq = SeqRecord(Seq('AGTAAGGGTTAGGAATA'))
dummy_seq2 = SeqRecord(Seq('AGTAAGGGTTAGGAATA'))

try:
    print('SeqRecord 객체 두 개 비교할 수 있니?')
    print(dummy_seq == dummy_seq2)
except Exception as e:
    print(f'아니 : {e}\n')
    
print(f'그럼 SeqRecord 객체 안 속성 값들은 비교할 수 있니? : {dummy_seq.seq == dummy_seq2.seq}')

SeqRecord 객체 두 개 비교할 수 있니?
아니 : SeqRecord comparison is deliberately not implemented. Explicitly compare the attributes of interest.

그럼 SeqRecord 객체 안 속성 값들은 비교할 수 있니? : True


# 2. FASTA 파일
- SeqIO 모듈로 FASTA 파일을 읽는 방법은 두 가지가 있다.  
(1) SeqIO.parse()  
(2) SeqIO.read()

## 2-1. SeqIO.parse() 메소드로 FASTA 파일 읽기
- SeqIO.parse() 메소드 안에는 2개의 인자를 입력으로 받는다.  
(1) 파일 이름  
(2) 유전체 데이터가 저장되어 있는 파일 종류  

&nbsp;(*) biopython 1.79 버전 기준 33개의 포맷을 지원하고 있다.  

- 자주 사용하는 데이터 포맷은 다음과 같다.
| 포맷 |  설명 |
|:----------:|:----------|
|clustal|Multiple alignment 툴에서 나온 결과로, 범용적으로 사용되는 alignment 포맷이다.|
|fasta|염기서열을 표기한 파일로 >로 시작하는 헤더와 서열이 있다. 헤더와 서열을 레코드라 하며, 1개 이상의 레코드가 저장되어 있다|
|fastq|FASTA 파일에 quality 정보가 추가된 파일 포맷이다.|
|genbank|염기서열 및 아미노산 서열, 종 정보, 관련 논문 저자, 제목, pubmed ID 등의 메타 데이터가 저장되어 있다.|



In [5]:
from Bio import SeqIO

sequences = SeqIO.parse('../dataset/fasta/sample_1.fasta', 'fasta')
print(f'읽어들인 fasta 파일의 타입 : {type(sequences)}\n')

for sequence in sequences:
    string_decoration(f'fasta 파일 안의 sequence record의 타입 : {type(sequence)}', num = 120)
    string_decoration(f'fasta 파일 안의 sequence record\n{sequence}', num = 120, idx = 2)

읽어들인 fasta 파일의 타입 : <class 'Bio.SeqIO.FastaIO.FastaIterator'>

------------------------------------------------------------------------------------------------------------------------
fasta 파일 안의 sequence record의 타입 : <class 'Bio.SeqRecord.SeqRecord'>
------------------------------------------------------------------------------------------------------------------------
fasta 파일 안의 sequence record
ID: AF501235.1
Name: AF501235.1
Description: AF501235.1 Influenzavirus A (A/duck/Shanghai/1/2000) hemagglutinin gene, complete cds
Number of features: 0
Seq('ATGGAGAAAATAGTGCTTCTTCTTGCAATAGTCAGTCTTGTTAAAAGTGATCAG...AGA') 
------------------------------------------------------------------------------------------------------------------------


In [132]:
sequences = SeqIO.parse('../dataset/fasta/sample_2.fasta', 'fasta')
print(f'읽어들인 fasta 파일의 타입 : {type(sequences)}\n')

for idx, sequence in enumerate(sequences, 1):
    string_decoration(f'fasta 파일 안의 sequence record{idx}의 타입 : {type(sequence)}', num=100)
    string_decoration(f'fasta 파일 안의 sequence record{idx}\n{sequence}', idx=2, num=100)
    print('\n' if idx == 1 else '')

읽어들인 fasta 파일의 타입 : <class 'Bio.SeqIO.FastaIO.FastaIterator'>

----------------------------------------------------------------------------------------------------
fasta 파일 안의 sequence record1의 타입 : <class 'Bio.SeqRecord.SeqRecord'>
----------------------------------------------------------------------------------------------------
fasta 파일 안의 sequence record1
ID: MH464856.1
Name: MH464856.1
Description: MH464856.1 Hepatitis B virus isolate MA134, complete genome
Number of features: 0
Seq('TTCCACAACATTCCACCAAGCTCTGCAGGATCCCAGAGTAAGAGGCCTGTATTT...GGG') 
----------------------------------------------------------------------------------------------------


----------------------------------------------------------------------------------------------------
fasta 파일 안의 sequence record2의 타입 : <class 'Bio.SeqRecord.SeqRecord'>
----------------------------------------------------------------------------------------------------
fasta 파일 안의 sequence record2
ID: CP002925.1
Name: CP002925.1
Descri

## 2-2. SeqIO.read() 메소드로 FASTA 파일 읽기
- SeqIO.read() 메소드도 SeqIO.parse() 메소드와 동일하게 두 개의 입력인자를 받는다.  
(1) 파일 이름  
(2) 유전체 데이터가 저장되어 있는 파일 종류  
- SeqIO.parse() 메소드로 파일을 읽으면 레코드 수 만큳 SeqRecord 객체가 iterable한 generator가 객체로 생성  
  SeqIO.read() 메소드로 파일을 읽으면 1개의 SeqRecord 객체가 만들어짐. (여러개의 레코드가 있는 경우에 에러 발생)

In [131]:
sequence = SeqIO.read('../dataset/fasta/sample_1.fasta', 'fasta')
string_decoration(f'읽어들인 fasta 파일의 타입 : {type(sequences)}', num=100)
string_decoration(f'fasta 파일 안의 sequence record\n{sequence}',num=100, idx=2)

try:
    print('\n레코드가 여러개인 파일 읽을 수 있니?')
    sequence = SeqIO.read('../dataset/fasta/sample_2.fasta', 'fasta')
except Exception as e:
    print(f'아니 : {e}') 

----------------------------------------------------------------------------------------------------
읽어들인 fasta 파일의 타입 : <class 'Bio.SeqIO.FastaIO.FastaIterator'>
----------------------------------------------------------------------------------------------------
fasta 파일 안의 sequence record
ID: AF501235.1
Name: AF501235.1
Description: AF501235.1 Influenzavirus A (A/duck/Shanghai/1/2000) hemagglutinin gene, complete cds
Number of features: 0
Seq('ATGGAGAAAATAGTGCTTCTTCTTGCAATAGTCAGTCTTGTTAAAAGTGATCAG...AGA') 
----------------------------------------------------------------------------------------------------

레코드가 여러개인 파일 읽을 수 있니?
아니 : More than one record found in handle


### (!) FASTA파일로 SeqRecord 객체 만들기
- SeqIO.parse() 메소드를 사용하면 FASTA 파일 안의 정보들을 SeqRecord 객체로 사용할 수 있다.  
(1) FASTA 파일의 헤더 부분이 파싱되어 SeqRecord 객체의 ID, Name, Description 속성으로 들어간다.  
(2) FASTA 파일에 저장되어 있는 염기 서열 정보는 SeqRecord 객체의 Seq 속성으로 들어간다.


In [130]:
lac_operon = SeqIO.parse('../dataset/fasta/J01631.1.fasta', 'fasta')
print(f'읽어들인 fasta 파일의 타입 : {type(lac_operon)}\n')

for lac in lac_operon:
    string_decoration(f'fasta 파일 안의 sequence record의 타입 : {type(lac)}', num = 120)
    string_decoration(f'fasta 파일 안의 sequence record\n{lac}', num = 120, idx = 2)
    print('\n')
    
    lac.name = '가나다라'
    string_decoration(f'임의로 바꾼 이름을 적용한 sequence record의 타입 : {type(lac)}', num = 120)
    string_decoration(f'임의로 바꾼 이름을 적용한 sequence record\n{lac}',num=120, idx=2)

읽어들인 fasta 파일의 타입 : <class 'Bio.SeqIO.FastaIO.FastaIterator'>

------------------------------------------------------------------------------------------------------------------------
fasta 파일 안의 sequence record의 타입 : <class 'Bio.SeqRecord.SeqRecord'>
------------------------------------------------------------------------------------------------------------------------
fasta 파일 안의 sequence record
ID: J01636.1
Name: J01636.1
Description: J01636.1 E.coli lactose operon with lacI, lacZ, lacY and lacA genes
Number of features: 0
Seq('GACACCATCGAATGGCGCAAAACCTTTCGCGGTATGGCATGATAGCGCCCGGAA...GAC') 
------------------------------------------------------------------------------------------------------------------------


------------------------------------------------------------------------------------------------------------------------
임의로 바꾼 이름을 적용한 sequence record의 타입 : <class 'Bio.SeqRecord.SeqRecord'>
------------------------------------------------------------------------------------

# 3. FASTQ 파일
- 일반적으로 FASTAQ 파일에는 여러개의 리드들로 구성되어 있어 SeqIO.read() 메소드로 읽으면 에러가 난다.

## 3-1. SeqIO.parse() 메소드로 FASTQ 파일 읽기

In [139]:
try:
    print('fastq 파일 읽을 수 있니?')
    sample_fastq = SeqIO.read('../dataset/fastq/sample_1.fastq', 'fastq')
except Exception as e:
    print(f'아니 : {e}\n')

sample_fastq = SeqIO.parse('../dataset/fastq/sample_1.fastq', 'fastq')
print(f'읽어들인 fastq 파일의 타입 : {type(sample_fastq)}')
for sample in sample_fastq:
    string_decoration(f'fastq 파일 안의 sequence 타입 : {type(sample)}', num = 120)
    string_decoration(f'fastq 파일 안의 sequence record\n{sample}',idx = 2, num=120)
    string_decoration(f'fastq 파일 안의 sequence \n{sample.seq}',idx = 2, num=120)
    print('\n')

fastq 파일 읽을 수 있니?
아니 : More than one record found in handle

읽어들인 fastq 파일의 타입 : <class 'Bio.SeqIO.QualityIO.FastqPhredIterator'>
------------------------------------------------------------------------------------------------------------------------
fastq 파일 안의 sequence 타입 : <class 'Bio.SeqRecord.SeqRecord'>
------------------------------------------------------------------------------------------------------------------------
fastq 파일 안의 sequence record
ID: SRR000982.5E745RJU01DDHJ6length=113
Name: SRR000982.5E745RJU01DDHJ6length=113
Description: SRR000982.5E745RJU01DDHJ6length=113
Number of features: 0
Per letter annotation for: phred_quality
Seq('AAGGCACCATGCAGAGATGCAAGGCCCCTTTCTAAGCCCTAGACTTCTGGATGA...CAG') 
------------------------------------------------------------------------------------------------------------------------
fastq 파일 안의 sequence 
AAGGCACCATGCAGAGATGCAAGGCCCCTTTCTAAGCCCTAGACTTCTGGATGACACTTCTAGAAACACCCTGGGCCAGAAGTGAACCTGCTGCCTTGAAGGGAATAACTCAG 
-------------------

## 3-2. SeqIO.parse() 메소드로 압축된 FASTQ 파일 읽기
- 연구에서 다루게 되는 FASTQ 파일은 대부분 GNU Zip(.gz) 파일 형태로 압축되어 있다.
- .gz 파일은 파이썬의 gzip 모듈을 사용하면 읽을 수 있다.

In [136]:
import gzip
sample_fastq = SeqIO.parse(gzip.open('../dataset/fastq/sample_1.fastq.gz', 'rt'), 'fastq')

for sample in sample_fastq:
    string_decoration(f'fastq 파일 안의 sequence 타입 : {type(sample)}', num = 120)
    string_decoration(f'fastq 파일 안의 sequence record\n{sample}',idx = 2, num=120)
    string_decoration(f'fastq 파일 안의 sequence \n{sample.seq}',idx = 2, num=120)
    print('\n')

------------------------------------------------------------------------------------------------------------------------
fastq 파일 안의 sequence 타입 : <class 'Bio.SeqRecord.SeqRecord'>
------------------------------------------------------------------------------------------------------------------------
fastq 파일 안의 sequence record
ID: SRR000982.5E745RJU01DDHJ6length=113
Name: SRR000982.5E745RJU01DDHJ6length=113
Description: SRR000982.5E745RJU01DDHJ6length=113
Number of features: 0
Per letter annotation for: phred_quality
Seq('AAGGCACCATGCAGAGATGCAAGGCCCCTTTCTAAGCCCTAGACTTCTGGATGA...CAG') 
------------------------------------------------------------------------------------------------------------------------
fastq 파일 안의 sequence 
AAGGCACCATGCAGAGATGCAAGGCCCCTTTCTAAGCCCTAGACTTCTGGATGACACTTCTAGAAACACCCTGGGCCAGAAGTGAACCTGCTGCCTTGAAGGGAATAACTCAG 
------------------------------------------------------------------------------------------------------------------------


--------------------------

# 4. Genbank 파일
- Genbank 파일은 FASTA 파일과 동일하게 SeqIO.parse(), SeqIO.read() 메소드로 읽을 수 있다.
- SeqIO.read()와 SeqIO.parse()를 이용하여 Genbank에서 필요한 정보를 가져올 수 있다.

|속성 이름|설명|특이 사항|
|:-----------:|:-----------|:-----------|
|id|GenBank 파일의 ID 정보를 말한다.|버전 정보를 포함한다.|
|seq|GenBank 파일의 서열 정보를 말한다.|-|
|name|GenBank 파일의 LOCUS 정보를 말한다.|-|
|description|GenBank 파일을 설명하는 내용이 들어간다.|-|
|dbxrefs|추가 db의 내용이 들어간다.|-|
|features|GenBank 파일의 각 REFERENCE 항목들이 들어간다.|-|
|annotations|GenBank 파일의 각 정보들이 들어간다.|파이썬 딕셔너리 형으로 들어옴.|
|letter_annotations|Genbank 파일에 추가 정보를 더 넣기 위해 만든 속성이다.|파이썬 딕셔너리 형으로 들어옴.|

In [179]:
sample_gbk = SeqIO.read('../dataset/genbank/KT225476.2.gbk', 'genbank')
string_decoration(f'SeqIO.read()로 읽어들인 genbank 파일의 타입 : {type(sample_gbk)}', num = 120)
string_decoration(f'genbank 파일 안의 sequence record\n{sample_gbk}',num=120, idx=2)
string_decoration(f'genbank 파일 안의 name \n{sample_gbk.name}',num=120, idx=2)
string_decoration(f'genbank 파일 안의 description \n{sample_gbk.description}',num=120, idx=2)
string_decoration(f'읽어들인 genbank 안의 sequence \n{sample_gbk.seq[:115]}', num = 120, idx = 2)
print('\n')

sample_gbk = SeqIO.parse('../dataset/genbank/KT225476.2.gbk', 'genbank')
string_decoration(f'SeqIO.parse()로 읽어들인 genbank 파일의 타입 : {type(sample_gbk)}', num = 120)
for sample in sample_gbk:
    string_decoration(f'genbank 파일 안의 sequence record\n{sample}',num=120, idx=2)
    string_decoration(f'읽어들인 genbank 안의 name \n{sample.name}', num = 120, idx = 2)
    string_decoration(f'읽어들인 genbank 안의 description \n{sample.description}', num = 120, idx = 2)
    string_decoration(f'읽어들인 genbank 안의 molecule type \n{sample.annotations["molecule_type"]}', num = 120, idx = 2)
    string_decoration(f'읽어들인 genbank 안의 organism \n{sample.annotations["organism"]}', num = 120, idx = 2)
    string_decoration(f'읽어들인 genbank 안의 sequence \n{sample.seq[:115]}', num = 120, idx = 2)

------------------------------------------------------------------------------------------------------------------------
SeqIO.read()로 읽어들인 genbank 파일의 타입 : <class 'Bio.SeqRecord.SeqRecord'>
------------------------------------------------------------------------------------------------------------------------
genbank 파일 안의 sequence record
ID: KT225476.2
Name: KT225476
Description: Middle East respiratory syndrome coronavirus isolate MERS-CoV/THA/CU/17_06_2015, complete genome
Number of features: 12
/molecule_type=RNA
/topology=linear
/data_file_division=VRL
/date=22-AUG-2017
/accessions=['KT225476']
/sequence_version=2
/keywords=['']
/source=Middle East respiratory syndrome-related coronavirus (MERS-CoV)
/organism=Middle East respiratory syndrome-related coronavirus
/taxonomy=['Viruses', 'ssRNA viruses', 'ssRNA positive-strand viruses, no DNA stage', 'Nidovirales', 'Coronaviridae', 'Coronavirinae', 'Betacoronavirus']
/references=[Reference(title='Imported case of Middle East respirato

# 5. Entrez 모듈
- NCBI(National Center for Biotechnology Information) 정보를 웹을 통해 접근하여 정보를 가져오는 모듈  
(!) 웹을 통해 접근하는 모듈이므로, 반드시 인터넷에 연결 되어있어야 한다.
- Entrez에는 사용자의 이메일이 의무적으로 들어간다.
- Entrez 모듈에서 가장 많이 사용하는 efetch 메소드를 이용한다.    
- Entrez.efetch()에는 키워드 인자로 아래와 같이 입력된다.  
(1) db : 데이터를 검색할 데이터 베이스 이름  
(2) id : 검색할 데이터의 id  
(3) rettype : 데이터 형식  
(4) retmode : 반환 형식  

In [207]:
from Bio import Entrez
Entrez.email = 'kimhippowork@gmail.com'

## 변이없는 CCR5 유전자 데이터를 GenBank 형식으로 받아오기
handle = Entrez.efetch(db='nucleotide', rettype='gb', id='AY463215', retmode='text')

print('genbank 파일 안의 sequece record')
for sample in handle:
    print(sample.strip())

genbank 파일 안의 sequece record
LOCUS       AY463215                1059 bp    DNA     linear   PRI 14-DEC-2004
DEFINITION  Homo sapiens CCR5 chemokine receptor (CCR5) gene, complete cds.
ACCESSION   AY463215
VERSION     AY463215.1
KEYWORDS    .
SOURCE      Homo sapiens (human)
ORGANISM  Homo sapiens
Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;
Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;
Catarrhini; Hominidae; Homo.
REFERENCE   1  (bases 1 to 1059)
AUTHORS   Capoulade-Metay,C., Ma,L., Truong,L.X., Dudoit,Y., Versmisse,P.,
Nguyen,N.V., Nguyen,M., Scott-Algara,D., Barre-Sinoussi,F.,
Debre,P., Bismuth,G., Pancino,G. and Theodorou,I.
TITLE     New CCR5 variants associated with reduced HIV coreceptor function
in southeast Asia
JOURNAL   AIDS 18 (17), 2243-2252 (2004)
PUBMED   15577536
REFERENCE   2  (bases 1 to 1059)
AUTHORS   Capoulade-Metay,C., Ma,L., Truong,L.X., Dudoit,Y., Versmisse,P.,
Nguyen,N.V., Nguyen,M., Scott-Algara,D., Barre-Sinoussi,F.,
Debre,

## 5-1. SeqIO를 이용한 Entrez 모듈로 받아온 정보 파싱

In [218]:
## 변이없는 CCR5 유전자 데이터를 FASTA 형식으로 받아오기
ccr5 = SeqIO.read(Entrez.efetch(db='nucleotide', rettype='fasta', id='AY463215', retmode='text'), 'fasta')
string_decoration(f'SeqIO.read()로 파싱한 Entrez의 CCR5 파일의 타입 : {type(ccr5)}', num = 120)
string_decoration(f'SeqIO.read()로 파싱한 Entrez의 CCR5 파일의 ID \n{ccr5.id}', idx=2, num = 120)
string_decoration(f'SeqIO.read()로 파싱한 Entrez의 CCR5 파일의 description \n{ccr5.description}', idx=2, num = 120)
string_decoration(f'SeqIO.read()로 파싱한 Entrez의 CCR5 파일의 sequence \n{ccr5.seq[:90]}, bp : {len(ccr5.seq)}', idx=2, num = 120)

------------------------------------------------------------------------------------------------------------------------
SeqIO.read()로 파싱한 Entrez의 CCR5 파일의 타입 : <class 'Bio.SeqRecord.SeqRecord'>
------------------------------------------------------------------------------------------------------------------------
SeqIO.read()로 파싱한 Entrez의 CCR5 파일의 ID 
AY463215.1 
------------------------------------------------------------------------------------------------------------------------
SeqIO.read()로 파싱한 Entrez의 CCR5 파일의 description 
AY463215.1 Homo sapiens CCR5 chemokine receptor (CCR5) gene, complete cds 
------------------------------------------------------------------------------------------------------------------------
SeqIO.read()로 파싱한 Entrez의 CCR5 파일의 sequence 
ATGGATTATCAAGTGTCAAGTCCAATCTATGACATCAATTATTATACATCGGAGCCCTGCCAAAAAATCAATGTGAAGCAAATCGCAGCC, bp : 1059 
------------------------------------------------------------------------------------------------------------------------

In [219]:
## 변이있는 CCR5 유전자 데이터를 fasta 형식으로 받아오기
ccr5_mutation = SeqIO.parse(Entrez.efetch(db='nucleotide', rettype='fasta', id='1575550', retmode='text'), 'fasta')
string_decoration(f'SeqIO.parse()로 파싱한 Entrez의 CCR5 파일의 타입 : {type(ccr5_mutation)}', num = 120)
for mutation in ccr5_mutation:
    string_decoration(f'SeqIO.parse()로 파싱한 Entrez의 변이 CCR5 파일의 ID \n{mutation.id}', idx=2, num = 120)
    string_decoration(f'SeqIO.read()로 파싱한 Entrez의 변이 CCR5 파일의 description \n{mutation.description}', idx=2, num = 120)
    string_decoration(f'SeqIO.read()로 파싱한 Entrez의 변이 CCR5 파일의 sequence \n{mutation.seq[:90]}, bp : {len(mutation.seq)}', idx=2, num = 120)

------------------------------------------------------------------------------------------------------------------------
SeqIO.parse()로 파싱한 Entrez의 CCR5 파일의 타입 : <class 'Bio.SeqIO.FastaIO.FastaIterator'>
------------------------------------------------------------------------------------------------------------------------
SeqIO.parse()로 파싱한 Entrez의 변이 CCR5 파일의 ID 
U66285.1 
------------------------------------------------------------------------------------------------------------------------
SeqIO.read()로 파싱한 Entrez의 변이 CCR5 파일의 description 
U66285.1 Human CC chemokine receptor CCR5 gene, mutant allele, complete cds 
------------------------------------------------------------------------------------------------------------------------
SeqIO.read()로 파싱한 Entrez의 변이 CCR5 파일의 sequence 
ATGGATTATCAAGTGTCAAGTCCAATCTATGACATCAATTATTATACATCGGAGCCCTGCCAAAAAATCAATGTGAAGCAAATCGCAGCC, bp : 1027 
-----------------------------------------------------------------------------------------------------

# 교재 연습문제를 풀어보자  
1. 제공한 GenBank 파일을 읽어 다음 질문에 답해보자  
    - 제공한 GenBank 파일의 바이러스는 어떤 종(organism) 인가?
    - 제공한 GenBank 파일의 annotation 속성에서 taxonomy를 출력해보자.
    - 바이러스 서열의 길이를 출력해보자.

In [32]:
genbank_seq = SeqIO.read('../dataset/genbank/HM624086.1.gbk', 'genbank')

string_decoration('genbank내의 SeqRecord')
string_decoration(genbank_seq, idx = 2)
print('\n')

string_decoration('답안', num=130)
string_decoration(f'genbank 내의 Organism : {genbank_seq.annotations["organism"]}, 길이 : {len(genbank_seq.seq)} (bp)', idx=2, num=130)
string_decoration(f'genbank 내의 taxonomy : {genbank_seq.annotations["taxonomy"]}', idx=3, num=130)

--------------------------------------------------------------------------------
genbank내의 SeqRecord
--------------------------------------------------------------------------------
ID: HM624086.1
Name: HM624086
Description: Influenza A virus (A/Perth/260/2009(H1N1)) segment 4 hemagglutinin (HA) gene, complete cds
Database cross-references: BioProject:PRJNA37813
Number of features: 3
/molecule_type=cRNA
/topology=linear
/data_file_division=VRL
/date=18-OCT-2011
/accessions=['HM624086']
/sequence_version=1
/keywords=['']
/source=Influenza A virus (A/Perth/260/2009(H1N1))
/organism=Influenza A virus (A/Perth/260/2009(H1N1))
/taxonomy=['Viruses', 'ssRNA viruses', 'ssRNA negative-strand viruses', 'Orthomyxoviridae', 'Influenzavirus A']
/references=[Reference(title='A comparison of pyrosequencing and neuraminidase inhibition assays for the detection of oseltamivir-resistant pandemic influenza A(H1N1) 2009 viruses', ...), Reference(title='Direct Submission', ...)]
/comment=Swine influenza A 

2. 관심 있는 종의 GenBank 파일을 바이오파이썬 Entrez 모듈을 사용하여 파싱하고 그 결과를 출력해보자

In [33]:
from Bio import Entrez
Entrez.email = 'kimhippowork@gmail.com'

In [38]:
## 거북이 torA
tortoise = SeqIO.read(Entrez.efetch(db='nucleotide', id='AF303221', rettype='gb', retmode='text'), 'genbank')

string_decoration('genbank내의 SeqRecord')
string_decoration(tortoise, idx = 2)
print('\n')

string_decoration('답안', num=130)
string_decoration(f'genbank 내의 Organism : {tortoise.annotations["organism"]}, 길이 : {len(tortoise.seq)} (bp)', idx=2, num=130)
string_decoration(f'genbank 내의 taxonomy : {tortoise.annotations["taxonomy"]}', idx=3, num=130)

--------------------------------------------------------------------------------
genbank내의 SeqRecord
--------------------------------------------------------------------------------
ID: AF303221.1
Name: AF303221
Description: Dictyostelium discoideum tortoise (torA) mRNA, complete cds; nuclear gene for mitochondrial product
Number of features: 3
/molecule_type=mRNA
/topology=linear
/data_file_division=INV
/date=23-FEB-2001
/accessions=['AF303221']
/sequence_version=1
/keywords=['']
/source=Dictyostelium discoideum
/organism=Dictyostelium discoideum
/taxonomy=['Eukaryota', 'Amoebozoa', 'Evosea', 'Eumycetozoa', 'Dictyostelia', 'Dictyosteliales', 'Dictyosteliaceae', 'Dictyostelium']
/references=[Reference(title='Tortoise, a novel mitochondrial protein, is required for directional responses of Dictyostelium in chemotactic gradients', ...), Reference(title='Direct Submission', ...)]
Seq('ATGAATAAAACAATAATAATAAATAATGTTTCAAAATATATAAAATTTAAAAAT...TAA') 
-----------------------------------------